In [105]:
from pathlib import Path
from buildingspy.io.outputfile import Reader
from datetime import timedelta, datetime
import pandas as pd

workdir = Path().resolve()
print(f"Working directory: {workdir}")

Working directory: /Users/nlong/working/urban-analysis/des-example-analysis/sdk_to_des_5G


In [112]:
def resample_and_convert_to_df(dymola_data, save_as_path):
    (time1, total_energy) = dymola_data.values("ETot.y")

    # Plant/pumps
    (_, sewer_pump) = dymola_data.values("pla.PPum")
    (_, ghx_pump) = dymola_data.values("pumSto.P")
    (_, distribution_pump) = dymola_data.values("pumDis.P")

    # buildings
    (_, ets_pump_1) = dymola_data.values("PPumETS.u[1]")
    (_, ets_pump_2) = dymola_data.values("PPumETS.u[2]")
    (_, ets_pump_3) = dymola_data.values("PPumETS.u[3]")
    (_, ets_hp_1) = dymola_data.values("PHeaPump.u[1]")
    (_, ets_hp_2) = dymola_data.values("PHeaPump.u[2]")
    (_, ets_hp_3) = dymola_data.values("PHeaPump.u[3]")

    # convert time to timestands for pandas
    time = [datetime(2022,1,1,0,0,0) + timedelta(seconds=int(t)) for t in time1]
    
    # convert into dataframe
    df_energy = pd.pandas.DataFrame({
        "datetime": time, 
        "total": total_energy
    })
    df_energy = df_energy.set_index('datetime')
    df_energy = df_energy.resample('60min').max()

    df_power = pd.pandas.DataFrame({
        "datetime": time, 
        "sewer_pump": sewer_pump,
        "ghx_pump": ghx_pump,
        "distribution_pump": distribution_pump,
        "ets_pump_1": ets_pump_1,
        "ets_pump_2": ets_pump_2,
        "ets_pump_3": ets_pump_3,
        "ets_hp_1": ets_hp_1,
        "ets_hp_2": ets_hp_2,
        "ets_hp_3": ets_hp_3,
    })
    df_power.to_csv(save_as_path / 'power_original.csv')
    df_power.drop_duplicates(subset='datetime', inplace=True)
    df_power = df_power.set_index('datetime')
    df_power.to_csv(save_as_path / 'power_deduped.csv')

    # upsample to 1min, without filling
    # df_power_1min = df_power.resample('1min').asfreq()
    # upsample to 1min with filling the last. This will
    # give us more accuracy on the energy use since it weights the power a bit more.
    df_power_1min = df_power.resample('1min').ffill()
    # df_power_1min.to_csv(save_as_path /'power_1min.csv')

    df_power_5min = df_power_1min.resample('5min').mean()
    df_power_5min.to_csv(save_as_path /'power_5min.csv')

    df_power_60min = df_power_5min.resample('60min').mean()
    df_power_60min.to_csv(save_as_path /'power_60min.csv')   

    return [df_power_60min, df_energy]



In [113]:
baseline_results = workdir / 'baseline' / 'baseline_5g.mat'
baseline_mat = Reader(baseline_results, 'dymola')

# all variables
# for var in mat.varNames():
#     if 'PPum' in var:
#         print(var)

high_eff_results = workdir / 'high_efficiency' / 'high_efficiency_5g.mat'
high_eff_mat = Reader(high_eff_results, 'dymola')

df_baseline_power, df_baseline_energy = resample_and_convert_to_df(baseline_mat, workdir / 'baseline') 
display(df_baseline_power)
df_he_power, df_he_energy = resample_and_convert_to_df(high_eff_mat, workdir / 'high_efficiency') 
display(df_he_power)

,sewer_pump,ghx_pump,distribution_pump,ets_pump_1,ets_pump_2,ets_pump_3,ets_hp_1,ets_hp_2,ets_hp_3
datetime,,,,,,,,,
2022-01-01 00:00:00,2347.094727,5579.373047,29210.568359,368.707367,1564.4198,683.826477,4161.563965,6011.191895,8146.833496
2022-01-01 01:00:00,2347.094727,5579.373047,29210.568359,101.218719,1564.4198,127.306038,599.850403,8592.489258,833.239929
2022-01-01 02:00:00,2347.094727,5579.373047,29210.568359,76.901566,1564.4198,76.713264,0.000000,8490.704102,0.000000
2022-01-01 03:00:00,2347.094727,5579.373047,29210.568359,76.901566,1564.4198,76.713264,0.000000,5229.400391,0.000000
2022-01-01 04:00:00,2347.094727,5579.373047,29210.568359,489.659088,1564.4198,647.355164,10325.109375,6709.659668,10642.346680
...,...,...,...,...,...,...,...,...,...
2022-12-31 20:00:00,2347.094727,5579.373047,29210.568359,92.964378,1564.4198,83.542519,0.000000,6992.812500,0.000000
2022-12-31 21:00:00,2347.094727,5579.373047,29210.568359,83.127708,1564.4198,643.108276,0.000000,9310.904297,1017.258545
2022-12-31 22:00:00,2347.094727,5579.373047,29210.568359,344.390228,1564.4198,683.826477,526.410095,10374.069336,7059.991699


,sewer_pump,ghx_pump,distribution_pump,ets_pump_1,ets_pump_2,ets_pump_3,ets_hp_1,ets_hp_2,ets_hp_3
datetime,,,,,,,,,
2022-01-01 00:00:00,2347.094727,5579.373047,29210.568359,397.458649,1707.836548,714.281250,5204.177246,4469.103516,8774.330078
2022-01-01 01:00:00,2347.094727,5579.373047,29210.568359,103.653137,1707.836548,129.845291,671.272888,6709.986816,894.263916
2022-01-01 02:00:00,2347.094727,5579.373047,29210.568359,76.943542,1707.836548,76.736427,0.000000,6088.855957,0.000000
2022-01-01 03:00:00,2347.094727,5579.373047,29210.568359,76.943542,1707.836548,76.736427,0.000000,4830.225586,0.000000
2022-01-01 04:00:00,2347.094727,5579.373047,29210.568359,519.184509,1707.836548,675.095764,11082.393555,5704.038574,11112.926758
...,...,...,...,...,...,...,...,...,...
2022-12-31 20:00:00,2347.094727,5579.373047,29210.568359,83.477264,1707.836548,81.641685,0.000000,5921.435547,0.000000
2022-12-31 21:00:00,2347.094727,5579.373047,29210.568359,79.426392,1707.836548,671.655945,0.000000,8432.319336,1421.258911
2022-12-31 22:00:00,2347.094727,5579.373047,29210.568359,370.749054,1707.836548,714.042725,1282.325562,9023.208008,7995.658203
